
Prioritizing certain workout types for different users with different goals.
The Prioritization tables are given as follows:

Priority Mapping of workouts for different goals:

Weight Loss	Cardio: 3	Strength: 2	Core: 2	Stretching: 1

Build Muscle	Cardio: 1	Strength: 3	Core: 2	Stretching: 1

Stay Fit	Cardio: 2	Strength: 2	Core: 2	Stretching; 2

Things to do next:-
1. Prioritize areas of focus for different workouts.(done)
2. Integrate it with the workouts filtered for intensity(user filtered workouts add workouts dataframe instead of workouts dataframe)(done)


In [3]:
!pip install pydatalog
from pyDatalog import pyDatalog

# Initialize terms
pyDatalog.create_terms('Age, Activity, MetRange, Difficulty, Min, Max')

# Define age group ranges
Min['adult'] = 15
Max['adult'] = 34
Min['middle_aged'] = 35
Max['middle_aged'] = 49
Min['older_adult'] = 50
Max['older_adult'] = 120

# MET range labels (string output, not numerical)
pyDatalog.create_terms('recommended_met, recommended_difficulty')

# Consistently lowercase difficulty terms
beginner = 'beginner'
intermediate = 'intermediate'

# --- ADULTS (15-34) ---
recommended_met(Age, 'low', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'high') <= (Min['adult'] <= Age) & (Age <= Max['adult'])

recommended_difficulty(Age, 'low', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'low', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])

# --- MIDDLE-AGED (35–49) ---
recommended_met(Age, Activity, 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_met(Age, 'high', 'moderate') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

recommended_difficulty(Age, Activity, beginner) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

# --- OLDER ADULTS (50+) ---
recommended_met(Age, Activity, 'low') <= (Age >= Min['older_adult']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Age >= Min['older_adult'])
recommended_met(Age, 'high', 'moderate') <= (Age >= Min['older_adult'])

recommended_difficulty(Age, Activity, beginner) <= (Age >= Min['older_adult'])


#Example Queries
print(recommended_met(18, 'low', MetRange).data)         # [('low',)]
print(recommended_difficulty(18, 'low', Difficulty).data)  # [('beginner',), ('intermediate',)]
print(recommended_met(40, 'moderate', MetRange).data)    # [('low',)]
print(recommended_difficulty(40, 'moderate', Difficulty).data)  # [('beginner',)]
print(recommended_met(60, 'high', MetRange).data)        # [('low',), ('moderate',)]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pydatalog: filename=pydatalog-0.17.4-cp311-cp311-linux_x86_64.whl size=1549123 sha256=27f69f5bfed705baa1eed973c37cff70d7bba21d527810243cc9cf86d8013b13
  Stored in directory: /root/.cache/pip/wheels/8f/f8/a0/c12514fe74e5f69d0d8967bd92363ea7a2961589ead1350b1d
Successfully built pydatalog
[('low',)]
[('intermediate',), ('beginner',)]
[('low',)]
[('beginner',)]
[('moderate',), ('low',)]


In [4]:
import pandas as pd

#Cleaning the diffiuclty column for trailing whitespaces
df = pd.read_csv("workouts.csv")
df['Difficulty'] = df['Difficulty'].str.strip().str.capitalize()

# MET Ranges mapping
MET_RANGES = {
    'low': (1.5, 3.9),
    'moderate': (4.0, 6.9),
    'high': (7.0, 10.0)
}

def get_user_input():
    return {
        'age': 45,
        'activity': 'high'
    }

def get_recommendations(age, activity):
    met_output = recommended_met(age, activity, MetRange).data
    diff_output = recommended_difficulty(age, activity, Difficulty).data

    met_recs = [item[0] for item in met_output]
    diff_recs = [item[0].capitalize() for item in diff_output]

    return met_recs, diff_recs

def filter_workouts(df, met_recs, diff_recs):
    met_mask = pd.Series([False] * len(df))
    for met_level in met_recs:
        if met_level in MET_RANGES:
            min_val, max_val = MET_RANGES[met_level]
            met_mask |= df['MET Value'].between(min_val, max_val)

    filtered_df = df[met_mask & df['Difficulty'].isin(diff_recs)]
    return filtered_df


def main():
    user = get_user_input()
    met_recs, diff_recs = get_recommendations(user['age'], user['activity'])

    print("Recommended MET Levels:", met_recs)
    print("Recommended Difficulties:", diff_recs)

    personalized_df = filter_workouts(df, met_recs, diff_recs)

    if not personalized_df.empty:
        print("\n Personalized Workout Recommendations:\n")
        print(personalized_df)
    else:
        print("\n No workouts match the given filters.")

    personalized_df.to_csv("filtered_workouts.csv", index=False)
main()


Recommended MET Levels: ['moderate', 'low']
Recommended Difficulties: ['Intermediate', 'Beginner']

 Personalized Workout Recommendations:

                              Name Target Muscle        Type  MET Value  \
0            Child's pose Stretch    Middle Back  Stretching        2.0   
1      Front and Back Neck Stretch   Middle Back  Stretching        2.0   
2             Lateral Neck Stretch   Middle Back  Stretching        2.0   
3                 Shoulder Circles    Quadriceps  Stretching        2.0   
4                    Ankle Circles        Calves    Mobility        2.3   
..                             ...           ...         ...        ...   
380           Lateral Bounding Hop    Quadriceps      Cardio        6.5   
381              Staggered push-up         Chest    Strength        6.5   
382                   Frog push-up         Chest    Strength        6.5   
383  Alternating Staggered Push-Up         Chest    Strength        6.5   
384                   Pike Push-Up 